# Introduction to Equation Discovery - Comparing Symbolic Regression Methods

Upto now we have seen that the climate models we developed are using physical equations that are based on our understanding of the physical processes that govern the climate. However, these equations are often complex and difficult to solve, and they can only be used to model the climate at a coarse resolution.

**Equation discovery is a different approach that uses machine learning to automatically discover equations that can be used to model the climate.** Specifically, equation discovery in climate modeling can be used to:
- **Automate the process of developing subgrid parameterizations.** Subgrid parameterizations are mathematical equations that are used to represent the effects of processes that are too small to be resolved by climate models. These equations are often complex and difficult to develop, but they can be automatically discovered using equation discovery.
- **Identify relationships between variables that are important for understanding climate change.** Machine learning algorithms can learn from large datasets of climate data to identify relationships between variables that are important for understanding climate change.
- **Develop models that are more interpretable and that can be used to make better decisions about how to mitigate and adapt to climate change.** Machine learning algorithms can generate equations that are easier to understand than traditional physical equations. This can help scientists and policymakers to understand how climate models work and to make better decisions about how to mitigate and adapt to climate change.

In this notebook, we'll review some common techniques for **Symbolic Regression (SR)**, a family of methods of discovering (simple) equations that relate inputs to outputs.

In particular, we'll cover the following methods:

- Symbolic regression based on {ref}`genetic-programming-section`
    * {ref}`gplearn-sec`
    * {ref}`pysr-sec`

- Symbolic regression based on {ref}`sparse-regression-sec`
    * {ref}`linear-regression-sec`
    * {ref}`lasso-sec`
    * {ref}`rvm-sec`
    * {ref}`stlsq-sec`

- Brief overview of other techniques

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pysindy as ps
from sklearn.linear_model import Lasso, LinearRegression
from sympy import Number, latex

from rvm import RVR

## Data

When explaining the methods, we'll consider the following bivariate system:

$$
y = x_0^2 - \frac{1}{2}x_1^2 + \sin\left(\frac{1}{2} x_0 x_1\right)
$$

Although the equation is relatively simple, discovering it from $x_0$ and $x_1$ directly is a bit tough, since the $\sin$ term requires going at least three levels deep in term combinations, and also involves an inner multiplication by a constant (which not all symbolic regression frameworks support).

In [ ]:
def toy_function(x):
    x0 = x[:, 0]
    x1 = x[:, 1]
    return x0**2 - 0.5 * x1**2 + np.sin(0.5 * x0 * x1)

In [ ]:
def contour(
    f, lines=None, linewidths=6, alpha=0.9, linestyles="solid", label=None, **kw
):
    grid = np.linspace(-4, 4, 150)
    xyg = np.array(np.meshgrid(grid, grid))
    X = xyg.reshape(2, -1).T
    zg = f(X)
    xg, yg = xyg
    vlim = np.abs(zg).max()
    plt.contourf(xg, yg, zg.reshape(xg.shape), 300, vmin=-vlim, vmax=vlim, cmap="bwr")
    plt.colorbar(label=label)

In [ ]:
x = np.random.normal(size=(1000, 2))
y = toy_function(x)

In [ ]:
plt.figure(dpi=150)
contour(toy_function, label="Function value")
plt.title("Toy domain for illustrating\nsymbolic regression", fontsize=16)
plt.xlabel("$x_0$", fontsize=16)
plt.ylabel("$x_1$", fontsize=16)
plt.show()

(genetic-programming-section)=
## Genetic Programming

Genetic programming is a classic technique in AI, and interest in the idea dates back at least to Alan Turing (who proposes the idea in the [same paper](https://doi.org/10.1093/mind/LIX.236.433) that introduces the imitation game, or Turing test).

The general idea is to have a:
- "Population" of programs (in our case, possible symbolic expressions, usually represented as trees)
- Some notion of individual "fitness" (in our case, how well they model the data, as well as their simplicity)
- Some means of applying random "mutations" (e.g. adding a term, deleting a term)

```{figure} figs/gp_mutation_types.png
:name: genetic-programming-mutation-types

Some examples for different mutation types
```

"Crossover" and "subtree" mutations combine features from multiple candidate equations, imitating the process of genetic recombination from sexual reproduction. "Hoist" and "point" mutations modify individual equations, imitating the process of genetic mutation from, e.g., random errors in DNA replication.

Specific mutations aside, programs can have a limited lifespan, and those with high "fitness" produce more "offspring" (possibly-mutated versions of themselves). However, everything is probabilistic, so programs with lower fitness can still persist. This is critically important, because it allows genetic programming to overcome local minima during optimization.

Note that although genetic programming can be a robust way to solve extremely difficult non-convex optimization problems (as perhaps evidenced by life itself), it also tends to be slow and resource-intensive.

(gplearn-sec)=
### gplearn

One nicely-documented library for "pure" genetic programming-based symbolic regression is [gplearn](https://gplearn.readthedocs.io/en/stable/), which provides a scikit-learn style API and a number of configuration options:

#### Run gplearn

In [ ]:
import gplearn
import gplearn.genetic

In [ ]:
gplearn_sr = gplearn.genetic.SymbolicRegressor(
    population_size=5000,
    generations=50,
    p_crossover=0.7,
    p_subtree_mutation=0.1,
    p_hoist_mutation=0.05,
    p_point_mutation=0.1,
    max_samples=0.9,
    verbose=1,
    parsimony_coefficient=0.01,
    function_set=("add", "mul", "sin"),
)
gplearn_sr.fit(x, y)

#### Interpret results

In [ ]:
print(gplearn_sr._program)

This looks very close to the right answer, though the constants are slightly off.

In [ ]:
[const for const in gplearn_sr._program.program if isinstance(const, float)]

This is likely because gplearn mutations which add or update constants always pick values by drawing random uniform values (within pre-specified ranges, by default -1 to 1). In my limited experience so far, this is one of the major inefficiencies.

(pysr-sec)=
### PySR

[PySR](https://pysr.readthedocs.io/en/latest/) is a different library for symbolic regression which, though still based on genetic programming, includes simulated annealing and gradient-free optimization to set the values of constants as well as performance improvements (although accessible via a Python API, the main library is written in Julia for speed). This seems to make it a bit more reliable and precise.

#### Run PySR

In [ ]:
import pysr

In [ ]:
equations = pysr.PySRRegressor(
    niterations=5,
    binary_operators=["+", "*"],  # operators that can combine two terms
    unary_operators=["sin"],  # operators that modify a single term
)
equations.fit(x, y)

#### Interpret results

In [ ]:
def round_expr(expr, num_digits=4):
    return expr.xreplace({n: round(n, num_digits) for n in expr.atoms(Number)})

In [ ]:
round_expr(equations.sympy())

It looks like PySR is able to not only discover the correct equation, but also format it for us nicely.

In addition, it saves a Pareto frontier of possible expressions of varying complexities, which had the lowest error of any other equations of equal or lesser complexity (with a configurable trade-off rule):

In [ ]:
equations.equations_

In [ ]:
equations.equations_.loss

In [ ]:
plt.figure(figsize=(12, 3), dpi=150)
plt.bar(
    np.arange(len(equations.equations_)),
    equations.equations_.loss,
    width=0.33,
    color="blue",
)


plt.yscale("log")
plt.ylabel("Mean squared error", fontsize=14, color="blue")
plt.xticks(
    range(len(equations.equations_)),
    [f"${latex(round_expr(v,7))}$" for v in equations.equations_.sympy_format],
    rotation=30,
    ha="right",
    fontsize=12,
)
plt.title("PySR Pareto frontier", fontsize=16)
plt.xlabel("Expression (more complex $\\to$)", fontsize=14)

ax2 = plt.twinx()
ax2.bar(
    np.arange(len(equations.equations_)) + 0.33,
    equations.equations_.score,
    width=0.33,
    color="orange",
)
ax2.set_ylabel("PySR score", color="orange", fontsize=14)

plt.show()

(sparse-regression-sec)=
## Manually Constructed Library + Sparse Regression

Another approach to performing symbolic regression is to
- Hand-construct a library of basis functions (e.g. by recursively combining all terms with various operations)
- Run sparse linear regression to extract a small set of basis functions

This approach works well for problems where any constants in the underlying formula can be extracted out to the outermost level (so they can be determined by linear regression). It also works well when we have domain knowledge about which terms could potentially appear in the final sum.

It does not work well in cases where constants can't be pulled outside certain operations (e.g. in this case, with the sine operator, though powers and derivatives are generally fine). This method can also struggle when basis function values are highly correlated, though it depends on the sparse regression method.

Let's create an augmentation function which takes an array of features and adds additional basis functions by combining them with binary multiplication and unary sine operations:

In [ ]:
def augment(feats, names):
    existing = set(names)
    new_feats = []
    new_names = []

    for i in range(feats.shape[1]):
        name = f"sin({names[i]})"
        if name not in existing:
            new_feats.append(np.sin(feats[:, i]))
            new_names.append(name)
        for j in range(i, feats.shape[1]):
            name = f"mul({names[i]}, {names[j]})"
            if name not in existing:
                new_feats.append(feats[:, i] * feats[:, j])
                new_names.append(name)

    return np.hstack((feats, np.array(new_feats).T)), names + new_names

In [ ]:
def repeatedly_augment(feats, names=None, depth=2):
    if names is None:
        names = [f"x{i}" for i in range(len(feats))]
    for _ in range(depth):
        feats, names = augment(feats, names)
    return feats, names

In [ ]:
feats, names = repeatedly_augment(np.array(x), ["x0", "x1"], depth=2)

Applying this twice brings us from 2 features up to 37:

In [ ]:
feats.shape

We can see that the complexity of this approach starts to blow up exponentially, though, as we increase the maximum depth of an expression:

In [ ]:
augment(feats, names)[0].shape

In [ ]:
augment(*augment(feats, names))[0].shape

If we need arbitrary polynomials of up to 4th order, we end up with orders of magnitude more features than samples, which will cause problems.

Note that since this method lacks the ability to apply operations with constants within other operations, $\sin(0.5 x_0 x_1)$ isn't present in any of these possible sets, though $\sin(x_0 x_1)$ is.

Let's now see how different methods for (sparse) linear regression perform on this task:

In [ ]:
def learned_expr(coef, names, cutoff=1e-2):
    coef_idx = np.argwhere(np.abs(coef) > cutoff)[:, 0]
    sort_order = reversed(np.argsort(np.abs(coef[coef_idx])))
    return " + ".join(
        [f"{coef[coef_idx[i]]:.4f}*{names[coef_idx[i]]}" for i in sort_order]
    )

(linear-regression-sec)=
### Linear regression

We can start with completely unregularized linear regression, which will find the linear combination of basis terms that minimizes mean squared error (to high precision, due to the convexity of the problem and an analytic way of solving it).

In [ ]:
linreg = LinearRegression()
linreg.fit(feats, y)
learned_expr(linreg.coef_, names)

Unfortunately, this doesn't seem to give us a very sparse solution in this case.

(lasso-sec)=
### L1-regularized linear regression (LASSO)

A common way to make linear regression return more sparse solutions is to minimize a linear combination of the mean squared error and the sum of the magnitudes of the weights. This is called applying an "L1 penalty" since it is the L1-norm of the weight vector, and is also referred to as the "Least Absolute Shrinkage and Selection Operator", or LASSO. From a Bayesian perspective, this corresponds to doing Bayesian linear regression with a Laplace or double-exponential prior on the weights.

Note that although this penalty does tend to encourage sparsity, it's also meant as a regularizer, which can help with the presence of noise. 

In [ ]:
lasso = Lasso(alpha=0.01)
lasso.fit(feats, y)
learned_expr(lasso.coef_, names)

Here, LASSO ends up doing a pretty good job finding the first two terms of the ground-truth model ($x_0^2 - 0.5x_1^2$), with an approximation of the remaining $\sin(0.5 x_0 x_1)$ term as $ 0.2(x_0\sin(x_1)+x_1\sin(x_0)) + 0.1(\sin(x_0 x_1) + x_0x_1)$:

In [ ]:
plt.figure(figsize=(4, 4), dpi=150)
x0 = x[:, 0]
x1 = x[:, 1]
sin = np.sin
plt.scatter(
    sin(0.5 * x0 * x1),
    0.2 * (x0 * sin(x1) + x1 * sin(x0)) + 0.1 * (sin(x0 * x1) + x0 * x1),
)
plt.plot([-1, 1], [-1, 1], color="red")
plt.xlabel(r"$\sin(0.5 x_0 x_1)$", fontsize=12)
plt.ylabel(
    r"$0.2(x_0\sin(x_1)+x_1\sin(x_0))$" + "\n" + r"$+ 0.1(\sin(x_0 x_1) + x_0x_1)$",
    fontsize=12,
)
plt.show()

(rvm-sec)=
### Relevance vector machines (RVM)

Relevance vector machines, used by [Zanna and Bolton 2020](https://laurezanna.github.io/files/Zanna-Bolton-2020.pdf) for equation discovery, provide another way of learning sparse linear models in a Bayesian manner, though with a different prior that prioritizes sparsity over shrinkage when the data permits it. RVMs can also be used with a kernel trick to learn nonlinear decision boundaries which are sparse in a kernelized feature space, though in this case, we use a linear kernel on top of our manually computed library features.

In [ ]:
rvm = RVR(verbose=False)
rvm.fit(feats, y, names)
learned_expr(rvm.m_, rvm.labels)

RVM does discover the $x_0^2$ and $-0.5x_1^2$ terms almost exactly, but ends up with an even more complex approximate expression for the missing $\sin(0.5x_0x_1)$ term.

(stlsq-sec)=
### Sequentially Thresholded Least Squares (STLSQ) from pysindy

The PySINDy library provides a number of efficient Python implementations of sparse regression algorithms. Their default regression method is sequentially thresholded least squares (STLSQ), which (as its name might suggest) repeatedly runs L2-regularized linear regression with penalty strength `alpha` and prunes features with weight magnitudes below a given `threshold`.

In [ ]:
stlsq = ps.optimizers.stlsq.STLSQ(alpha=0.0001, threshold=0.1)
stlsq.fit(feats, y)
learned_expr(stlsq.coef_[0], names)

This method ends up finding a sparser solution than Lasso, though I did tweak the parameters a bit to make that happen :)

It's worth checking out [PySINDy's full suite of sparse regression methods](https://pysindy.readthedocs.io/en/latest/api/pysindy.optimizers.html) for other approaches!

Overall, all of these methods were able to find the $x_0^2$ and $-0.5x_1^2$ terms, but all of them also needed to find approximations for the remaining term because it wasn't in the feature library. Meaning the expressions are both less accurate and more complex.

### Tweaking the Feature Library

Now, we can potentially resolve some of these issues by pre-multiplying $x_1$ by 0.5 before the augmentation process (which we'll call $x_{1h}$).

In [ ]:
feats2 = np.array(x)
feats2[:, 1] = feats2[:, 1] * 0.5
names2 = ["x0", "x1h"]
depth = 2

for _ in range(depth):
    feats2, names2 = augment(feats2, names2)

In this case, the correct model should be expressed as

$$
y = x_0^2 - 2x_{1h}^2 + \sin(x_0 x_{1h})
$$

which is now a linear combination of our basis terms. Let's see if these regression techniques can find it:

In [ ]:
linreg = LinearRegression()
linreg.fit(feats2, y)
learned_expr(linreg.coef_, names2)

In [ ]:
lasso = Lasso(alpha=0.01)
lasso.fit(feats2, y)
learned_expr(lasso.coef_, names2)

In [ ]:
rvm = RVR(verbose=False)
rvm.fit(feats2, y, names2)
learned_expr(rvm.m_, rvm.labels)

In [ ]:
stlsq = ps.optimizers.stlsq.STLSQ(alpha=0.0001, threshold=0.1)
stlsq.fit(feats2, y)
learned_expr(stlsq.coef_[0], names2)

In this case, every method except Lasso finds the true model exactly (which held true across many choices of regularization parameter).

## Handling noise

What if we add noise to the data?

In [ ]:
noise = np.random.normal(size=feats2.shape) * 0.01

In [ ]:
linreg = LinearRegression()
linreg.fit(feats2 + noise, y)
learned_expr(linreg.coef_, names2)

In [ ]:
lasso = Lasso(alpha=0.01)
lasso.fit(feats2 + noise, y)
learned_expr(lasso.coef_, names2)

In [ ]:
rvm = RVR(verbose=False)
rvm.fit(feats2 + noise, y, names2)
learned_expr(rvm.m_, rvm.labels)

In [ ]:
stlsq = ps.optimizers.stlsq.STLSQ(alpha=0.0001, threshold=0.1)
stlsq.fit(feats2 + noise, y)
learned_expr(stlsq.coef_[0], names2)

In this case, it actually becomes _Lasso_ which gets closest to the true model (i.e. is the only regression technique whose first three leading terms match the ground-truth model). This reversal illustrates how noise can strongly impact the effectiveness of different methods for learning symbolic models.

Let's see how genetic programming-based techniques deal with noise.

In [ ]:
noisy_equations = pysr.PySRRegressor(
    niterations=5,
    binary_operators=["+", "*"],  # operators that can combine two terms
    unary_operators=["sin"],  # operators that modify a single term
)
noisy_equations.fit(x, y + np.random.normal(size=y.shape));

In [ ]:
noisy_equations.equations_

In [ ]:
round_expr(noisy_equations.sympy())

In [ ]:
noisy_equations.equations_.sympy_format.values[-4]

It looks like PySR was still able to discover the true expression as part of its Pareto frontier, though with the noise it's only rated third-best for the default tradeoff between complexity and performance (with a sin-less version taking first).

## Other Methods for Symbolic Regression

To finish the notebook, here are a few other techniques for symbolic regression that don't fit under either previously described methods (and all developed in the last several years):

- [STreCH](https://arxiv.org/pdf/2102.08351.pdf) (Julia implementation [here](https://github.com/jongeunkim/STreCH)) formulates symbolic regression as a non-convex mixed-integer nonlinear programming (MINLP) technique, which it solves using a heuristic-guided cutting plane formulation.
- [EQL](http://proceedings.mlr.press/v80/sahoo18a/sahoo18a.pdf) (Python implementation [here](https://github.com/martius-lab/EQL)) develops "equation layers" for neural networks which can be readily interpreted and also chained together, and are regularized with a thresholded L1 penalty to encourage sparsity.
- [AI Feynman](https://arxiv.org/abs/2006.10782) (hybrid Python-Fortran implementation [here](https://github.com/SJ001/AI-Feynman)) starts by training a neural network, then computes _input gradients_ of the learned network, and then uses attributes of those gradients to decompose the symbolic regression problem into something more tractable using graph theory.
- [Symbolic Metamodeling](https://papers.nips.cc/paper/2019/hash/567b8f5f423af15818a068235807edc0-Abstract.html) (Python implementation [here](https://github.com/ahmedmalaa/Symbolic-Metamodeling)) uses gradient descent to learn compositions of Meijer-G functions, a flexible family that can be programmatically projected back onto a wide set of analytic, closed-form, or even algebriac expressions.